In [11]:
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import pandas as pd
import numpy as np 
import openpyxl
import sklearn.linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier  
from sklearn.ensemble import RandomForestRegressor  
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn import svm
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import re
import statsmodels.api as sm
import os

url = 'https://fbref.com/en/comps/13/schedule/Ligue-1-Scores-and-Fixtures'
page = requests.get(url)
html_content = page.content
soup = BeautifulSoup(html_content, 'html.parser')
table = soup.find('table')
df = pd.read_html(str(table))[0].dropna(subset = 'Wk').reset_index(drop=True)
df['Wk'] = df['Wk'].astype(int)
 
affiches = df[df['Score'].isna()]
first = affiches['Wk'].min()

df = df[df['Wk'] <= first]
df = df[['Wk', 'Home', 'Score', 'Away']]
noms_colonnes = ['Journée','Domicile','Extérieur','Buts domicile','Buts extérieur','Résultat']
df['Buts domicile'] = df['Score'].apply(lambda x: int(str(x)[0]) if str(x)[0].isdigit() else 0)
df['Buts extérieur'] = df['Score'].apply(lambda x: int(str(x)[2]) if str(x)[2].isdigit() else 0)

df.drop(['Score'], axis = 1, inplace = True)
df['Résultat'] = -1
df.loc[df['Buts domicile'] > df['Buts extérieur'], 'Résultat'] = 1
df.loc[df['Buts domicile'] == df['Buts extérieur'], 'Résultat'] = 0
df.columns = noms_colonnes

df.insert(3, 'Equipe 1 à Domicile', 1)
noms_colonnes = ['Journée', 'Equipe 1', 'Equipe 2', 'Equipe 1 à Domicile', 'Buts Equipe 1', 'Buts Equipe 2', 'Résultat']
df.columns = noms_colonnes
df_copie = df.copy()[['Journée', 'Equipe 2', 'Equipe 1', 'Equipe 1 à Domicile', 'Buts Equipe 2', 'Buts Equipe 1', 'Résultat']]
df_copie.columns = noms_colonnes
df_copie['Equipe 1 à Domicile'] = 0
df = df._append(df_copie, ignore_index=True)
df = df.sort_values(by=['Equipe 1', 'Journée']).reset_index(drop=True)
conditions = [
    (df['Buts Equipe 1'] > df['Buts Equipe 2']),
    (df['Buts Equipe 1'] < df['Buts Equipe 2'])
]
valeurs = [1, -1]
df['Résultat'] = 0
df['Résultat'] = np.select(conditions, valeurs)

df['GF'] = df['Buts Equipe 1'].astype(float).astype(int)
df['GA'] = df['Buts Equipe 2'].astype(float).astype(int)

# Ajout de colonnes pour la différence de buts
df['GD'] = df['GF']- df['GA']
df['Points'] = df['Résultat'].map({1: 3, 0: 1, -1: 0})

df.sort_values(by=['Journée', 'Equipe 1']).reset_index()
result = df.groupby(['Equipe 1']).agg({
    'Points': 'cumsum',
    'GD': 'cumsum',
    'GF': 'cumsum',
    'GA' : 'cumsum'
}).reset_index()

df[['CPoints', 'CGD', 'CGF', 'CGA']] = result[['Points', 'GD', 'GF', 'GA']]
df = df.sort_values(by=['Journée', 'CPoints', 'CGD', 'CGF'], ascending=[True, False, False, False]).reset_index()
df['Classement Equipe 1'] = df.groupby(['Journée']).cumcount() + 1
df['Classement Equipe 1'] = df.groupby(['Equipe 1'])['Classement Equipe 1'].shift(1).astype(pd.Int64Dtype())

df = df.sort_values(by=['Equipe 1', 'Journée']).reset_index(drop=True).drop(['index','Points', 'GD', 'GF', 'GA','CPoints', 'CGD', 'CGF', 'CGA'], axis=1)
df['Classement Equipe 2'] = df.merge(df, how='left', left_on=['Journée', 'Equipe 1'], right_on=['Journée', 'Equipe 2'])['Classement Equipe 1_y']
df['Moyenne_BM par 1'] = df.groupby(['Equipe 1'])['Buts Equipe 1'].transform(lambda x: x.shift(1).expanding().mean())
df['Moyenne_BM par 2'] = df.merge(df, how='left', left_on=['Journée', 'Equipe 1'], right_on=['Journée', 'Equipe 2'])['Moyenne_BM par 1_y']
df.loc[df['Equipe 1 à Domicile'] == 1, 'Moyenne_BM par 1 à Domicile'] = (
    (df.groupby(['Equipe 1', 'Equipe 1 à Domicile'])['Buts Equipe 1'].cumsum() - df['Buts Equipe 1']) / 
    (df.groupby(['Equipe 1', 'Equipe 1 à Domicile'])['Journée'].cumcount())
)
df.loc[df['Equipe 1 à Domicile'] == 0, 'Moyenne_BM par 1 à Domicile'] = (
    (df.groupby(['Equipe 1'])['Buts Equipe 1'].cumsum() - df.groupby(['Equipe 1', 'Equipe 1 à Domicile'])['Buts Equipe 1'].cumsum()) 
    / (df.groupby(['Equipe 1'])['Journée'].cumcount() - df.groupby(['Equipe 1', 'Equipe 1 à Domicile'])['Journée'].cumcount())
)
df['Moyenne_BM par 2 à Domicile'] = df.merge(df, how='left', left_on=['Journée', 'Equipe 1'], right_on=['Journée', 'Equipe 2'])['Moyenne_BM par 1 à Domicile_y']
df.loc[df['Equipe 1 à Domicile'] == 0, 'Moyenne_BM par 1 à Extérieur'] = (
    (df.groupby(['Equipe 1', 'Equipe 1 à Domicile'])['Buts Equipe 1'].cumsum() - df['Buts Equipe 1']) 
    / (df.groupby(['Equipe 1', 'Equipe 1 à Domicile'])['Journée'].cumcount())
)
df.loc[df['Equipe 1 à Domicile'] == 1, 'Moyenne_BM par 1 à Extérieur'] = (
    (df.groupby(['Equipe 1'])['Buts Equipe 1'].cumsum() - df.groupby(['Equipe 1', 'Equipe 1 à Domicile'])['Buts Equipe 1'].cumsum()) 
    / (df.groupby(['Equipe 1'])['Journée'].cumcount() - df.groupby(['Equipe 1', 'Equipe 1 à Domicile'])['Journée'].cumcount())
)
df['Moyenne_BM par 2 à Extérieur'] = df.merge(df, how='left', left_on=['Journée', 'Equipe 1'], right_on=['Journée', 'Equipe 2'])['Moyenne_BM par 1 à Extérieur_y']
df['Moyenne_BE par 1'] = df.groupby(['Equipe 1'])['Buts Equipe 2'].transform(lambda x: x.shift(1).expanding().mean())
df['Moyenne_BE par 2'] = df.merge(df, how='left', left_on=['Journée', 'Equipe 1'], right_on=['Journée', 'Equipe 2'])['Moyenne_BE par 1_y']
df.loc[df['Equipe 1 à Domicile'] == 1, 'Moyenne_BE par 1 à Domicile'] = (
    (df.groupby(['Equipe 1', 'Equipe 1 à Domicile'])['Buts Equipe 2'].cumsum() - df['Buts Equipe 2']) 
    / (df.groupby(['Equipe 1', 'Equipe 1 à Domicile'])['Journée'].cumcount())
)
df.loc[df['Equipe 1 à Domicile'] == 0, 'Moyenne_BE par 1 à Domicile'] = (
    (df.groupby(['Equipe 1'])['Buts Equipe 2'].cumsum() - df.groupby(['Equipe 1', 'Equipe 1 à Domicile'])['Buts Equipe 2'].cumsum()) 
    / (df.groupby(['Equipe 1'])['Journée'].cumcount() - df.groupby(['Equipe 1', 'Equipe 1 à Domicile'])['Journée'].cumcount())
)
df['Moyenne_BE par 2 à Domicile'] = df.merge(df, how='left', left_on=['Journée', 'Equipe 1'], right_on=['Journée', 'Equipe 2'])['Moyenne_BE par 1 à Domicile_y']
df.loc[df['Equipe 1 à Domicile'] == 0, 'Moyenne_BE par 1 à Extérieur'] = (
    (df.groupby(['Equipe 1', 'Equipe 1 à Domicile'])['Buts Equipe 2'].cumsum() - df['Buts Equipe 2']) 
    / (df.groupby(['Equipe 1', 'Equipe 1 à Domicile'])['Journée'].cumcount())
)
df.loc[df['Equipe 1 à Domicile'] == 1, 'Moyenne_BE par 1 à Extérieur'] = (
    (df.groupby(['Equipe 1'])['Buts Equipe 2'].cumsum() - df.groupby(['Equipe 1', 'Equipe 1 à Domicile'])['Buts Equipe 2'].cumsum()) 
    / (df.groupby(['Equipe 1'])['Journée'].cumcount() - df.groupby(['Equipe 1', 'Equipe 1 à Domicile'])['Journée'].cumcount()))
df['Moyenne_BE par 2 à Extérieur'] = df.merge(df, how='left', left_on=['Journée', 'Equipe 1'], right_on=['Journée', 'Equipe 2'])['Moyenne_BE par 1 à Extérieur_y']
df['Forme 1'] = df.groupby(['Equipe 1'])['Résultat'].rolling(window=6, min_periods=2).sum().reset_index(drop=True)-df['Résultat']
df['Forme 2'] = df.merge(df, how='left', left_on=['Journée', 'Equipe 1'], right_on=['Journée', 'Equipe 2'])['Forme 1_y']
df['Historique'] = df.groupby(['Equipe 1', 'Equipe 2'])['Résultat'].cumsum() - df['Résultat']



df2 = pd.read_csv('C:\\Users\\vtgra\\Desktop\\Projet python\\dataframe_final.csv',encoding = 'utf-8')
df2 = pd.concat([df2,df[df['Journée'] <= first-1]])
df2['Saison'] = df2['Saison'].fillna('2023-2024')
df2['Classement Equipe 1'] = df2['Classement Equipe 1'].astype(pd.Int64Dtype())
df2['Classement Equipe 2'] = df2['Classement Equipe 2'].astype(pd.Int64Dtype())

df2




c:\Users\vtgra\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:447: RuntimeWarning: invalid value encountered in cast
  data = self._data.astype(dtype.numpy_dtype, copy=copy)


,Saison,Journée,Equipe 1,Equipe 2,Equipe 1 à Domicile,Buts Equipe 1,Buts Equipe 2,Résultat,Classement Equipe 1,Classement Equipe 2,...,Moyenne_BM par 2 à Extérieur,Moyenne_BE par 1,Moyenne_BE par 2,Moyenne_BE par 1 à Domicile,Moyenne_BE par 2 à Domicile,Moyenne_BE par 1 à Extérieur,Moyenne_BE par 2 à Extérieur,Forme 1,Forme 2,Historique
0,2002-2003,1,Ajaccio,Strasbourg,0,1,1,0,<NA>,<NA>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2002-2003,2,Ajaccio,Guingamp,1,0,2,-1,9,7,...,NaN,1.000000,3.000000,NaN,3.000000,1.000000,NaN,0.0,0.0,0
2,2002-2003,3,Ajaccio,Paris S-G,0,2,2,0,16,6,...,0.000000,1.500000,0.000000,2.000000,0.000000,1.000000,0.000000,-1.0,1.0,0
3,2002-2003,4,Ajaccio,Sedan,1,1,0,1,15,19,...,1.000000,1.666667,2.333333,2.000000,0.500000,1.500000,6.000000,-1.0,-2.0,0
4,2002-2003,5,Ajaccio,Troyes,0,0,1,-1,13,17,...,0.000000,1.250000,1.500000,1.000000,3.000000,1.500000,0.000000,0.0,-2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,2023-2024,13,Toulouse,Nice,0,0,1,-1,15,2,...,1.166667,1.333333,0.333333,1.200000,0.166667,1.428571,0.500000,-2.0,4.0,0
319,2023-2024,14,Toulouse,Lorient,1,1,1,0,15,16,...,1.000000,1.307692,1.846154,1.200000,1.571429,1.375000,2.166667,-3.0,-2.0,0
320,2023-2024,15,Toulouse,Lyon,0,0,3,-1,15,18,...,0.571429,1.285714,1.928571,1.166667,2.285714,1.375000,1.571429,-3.0,-2.0,0
321,2023-2024,16,Toulouse,Rennes,1,0,0,0,15,13,...,0.333333,1.400000,1.400000,1.166667,1.555556,1.555556,1.166667,-3.0,-3.0,0


In [15]:
print(pd.read_csv('Fbref_alex\\SOCCER_241223.csv'))

      Unnamed: 0        Date   Time                   Comp  \
0              0  2018-07-26  17:45              Europa Lg   
1              1  2018-08-02  20:45              Europa Lg   
2              2  2018-08-04  20:00  Trophée des Champions   
3              3  2018-08-04  20:00  Trophée des Champions   
4              4  2018-08-09  20:00              Europa Lg   
...          ...         ...    ...                    ...   
4550        4550  2023-12-20  21:00                Ligue 1   
4551        4551  2023-12-20  21:00                Ligue 1   
4552        4552  2023-12-20  21:00                Ligue 1   
4553        4553  2023-12-20  21:00                Ligue 1   
4554        4554  2023-12-20  21:00                Ligue 1   

                        Round  Day    Venue Result   GF   GA  ...  \
0     Second qualifying round  Thu     Away      W    1    0  ...   
1     Second qualifying round  Thu     Home      W    2    1  ...   
2       Trophée des Champions  Sat  Neutral     

In [7]:
dataframe_regression = df2.dropna().copy()

dataframe_regression['Classement Equipe 1'] = 1/dataframe_regression['Classement Equipe 1']
dataframe_regression['Classement Equipe 2'] = 1/dataframe_regression['Classement Equipe 2']
dataframe_regression[['BM Equipe 1 à D si D', 'BM Equipe 1 à E si E', 'BE Equipe 1 à D si D', 'BE Equipe 1 à E si E', 'BM Equipe 2 à D si D',
                      'BM Equipe 2 à E si E', 'BE Equipe 2 à D si D', 'BE Equipe 2 à E si E']] = dataframe_regression.apply(
        lambda row: [row['Equipe 1 à Domicile'] * row['Moyenne_BM par 1 à Domicile'], (1 - row['Equipe 1 à Domicile']) * row['Moyenne_BM par 1 à Extérieur'], 
                     row['Equipe 1 à Domicile'] * row['Moyenne_BE par 1 à Domicile'], (1 - row['Equipe 1 à Domicile']) * row['Moyenne_BE par 1 à Extérieur'],
                     (1-row['Equipe 1 à Domicile']) * row['Moyenne_BM par 2 à Domicile'], (row['Equipe 1 à Domicile']) * row['Moyenne_BM par 2 à Extérieur'], 
                     (1-row['Equipe 1 à Domicile']) * row['Moyenne_BE par 1 à Domicile'], (row['Equipe 1 à Domicile']) * row['Moyenne_BE par 1 à Extérieur']], axis=1, result_type='expand')

X_train = dataframe_regression.drop(['Equipe 1', 'Equipe 2', 'Saison', 'Journée', 'Buts Equipe 1', 'Buts Equipe 2', 'Résultat'], axis=1)
X_train['poids'] = np.where(dataframe_regression['Journée'] > 15, 2, 1)
weights_train = X_train['poids']
X_train = X_train.drop(['poids'], axis=1)
X_train1 = sm.add_constant(X_train)
Y_train = dataframe_regression["Buts Equipe 1"]
Y_train2 = dataframe_regression["Résultat"]


random_forest_model = RandomForestClassifier(n_estimators=1000, random_state=5)
random_forest_model.fit(X_train, Y_train2)

random_forest_model2 = RandomForestClassifier(n_estimators=1000, random_state=5)
random_forest_model2.fit(X_train, Y_train)

model1 = sm.WLS(Y_train.astype(float), X_train1.astype(float), weights = weights_train)
results1 = model1.fit()

model2 = sm.WLS(Y_train2.astype(float), X_train1.astype(float), weights = weights_train)
results2 = model2.fit()

svm_model = svm.SVC(kernel='rbf', C=50, random_state=42)
svm_model.fit(X_train, Y_train2)


SVC(C=50, random_state=42)

In [10]:
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import os

df_int = df.copy()
df_int['Résultat'] = df_int['Résultat'].replace({1: 'V', 0: 'N', -1: 'D'})
def trouver_chemins_images_avec_mot_cle(dossier, mot_cle):
    chemins_images = []

    fichiers = os.listdir(dossier)

    for fichier in fichiers:
        chemin_fichier = os.path.join(dossier, fichier)

        if os.path.isfile(chemin_fichier) and fichier.lower().endswith(('.png', '.jpg', '.jpeg', '.gif')):
            
            if mot_cle.lower() in fichier.lower():
                chemins_images.append(chemin_fichier)
                print(f"Chemin de l'image trouvée : {chemin_fichier}")
    return chemins_images[0]

photo1 = None
photo2 = None
first = affiches['Wk'].min()

def fonction_journées(event):
    sélection = int(journées_combobox.get()[1:])
    matchs = [affiche for affiche, wk in zip(affiches['Home'] + ' - ' + affiches['Away'], affiches['Wk']) if wk == sélection]
    matchs_combobox['values'] = matchs
    matchs_combobox.set("Sélectionnez un match")

def fonction_stats(event):
    global photo1, photo2, equipe1, equipe2
    equipe1 = matchs_combobox.get().split(' - ')[0].strip()
    equipe2 = matchs_combobox.get().split(' - ')[1].strip()
    noms_colonnes1 = ['Classement', 'Moyenne buts marqués', 'Moyenne buts marqués à Domicile', 'Moyenne buts encaissés', 'Moyenne buts encaissés à Domicile', 'Forme', 'Historique']
    noms_colonnes2 = ['Classement', 'Moyenne buts marqués', 'Moyenne buts marqués à Extérieur', 'Moyenne buts encaissés', 'Moyenne buts encaissés à Extérieur', 'Forme', 'Historique']
    df_match1 = df[((df['Equipe 1'] == equipe1) & (df['Journée'] == first))][['Classement Equipe 1', 'Moyenne_BM par 1','Moyenne_BE par 1', 'Moyenne_BM par 1 à Domicile', 'Moyenne_BE par 1 à Domicile']].reset_index(drop=True)
    df_match2 = df[((df['Equipe 2'] == equipe2) & (df['Journée'] == first))][[ 'Classement Equipe 2', 'Moyenne_BM par 2', 'Moyenne_BE par 2', 'Moyenne_BM par 2 à Extérieur', 'Moyenne_BE par 2 à Extérieur']].reset_index(drop=True)
    columns_to_round = ['Moyenne_BM par 1', 'Moyenne_BM par 1 à Domicile', 'Moyenne_BE par 1', 'Moyenne_BE par 1 à Domicile']
    df_match1[columns_to_round] = df_match1[columns_to_round].round(2)

    columns_to_round = ['Moyenne_BM par 2', 'Moyenne_BM par 2 à Extérieur', 'Moyenne_BE par 2', 'Moyenne_BE par 2 à Extérieur']
    df_match2[columns_to_round] = df_match2[columns_to_round].round(2)

    df_match1['Forme'] = (str(df_int[((df_int['Equipe 1'] == equipe1) & (df_int['Journée'] == first - 1))]['Résultat'].iloc[0]) + 
                          str(df_int[((df_int['Equipe 1'] == equipe1) & (df_int['Journée'] == first - 2))]['Résultat'].iloc[0]) + 
                          str(df_int[((df_int['Equipe 1'] == equipe1) & (df_int['Journée'] == first - 3))]['Résultat'].iloc[0]) + 
                          str(df_int[((df_int['Equipe 1'] == equipe1) & (df_int['Journée'] == first - 4))]['Résultat'].iloc[0]) + 
                          str(df_int[((df_int['Equipe 1'] == equipe1) & (df_int['Journée'] == first - 5))]['Résultat'].iloc[0])
    )

    df_match2['Forme'] = (str(df_int[((df_int['Equipe 1'] == equipe2) & (df_int['Journée'] == first - 1))]['Résultat'].iloc[0]) + 
                          str(df_int[((df_int['Equipe 1'] == equipe2) & (df_int['Journée'] == first - 2))]['Résultat'].iloc[0]) + 
                          str(df_int[((df_int['Equipe 1'] == equipe2) & (df_int['Journée'] == first - 3))]['Résultat'].iloc[0]) + 
                          str(df_int[((df_int['Equipe 1'] == equipe2) & (df_int['Journée'] == first - 4))]['Résultat'].iloc[0]) + 
                          str(df_int[((df_int['Equipe 1'] == equipe2) & (df_int['Journée'] == first - 5))]['Résultat'].iloc[0])
    )
    try:
        a = str(int(df2[(df2['Equipe 1'] == equipe1) & (df2['Equipe 2'] == equipe2)]['Résultat'].value_counts().add(df[(df['Equipe 1'] == equipe1) & (df['Equipe 2'] == equipe2)]['Résultat'].value_counts(),fill_value=0)[1]))
    except Exception as e:
        a = '0'
    try:
        b = str(int(df2[(df2['Equipe 1'] == equipe1) & (df2['Equipe 2'] == equipe2)]['Résultat'].value_counts().add(df[(df['Equipe 1'] == equipe1) & (df['Equipe 2'] == equipe2)]['Résultat'].value_counts(),fill_value=0)[-1]))
    except Exception as e:
        b = '0'
    try:
        c = str(int(df2[(df2['Equipe 1'] == equipe1) & (df2['Equipe 2'] == equipe2)]['Résultat'].value_counts().add(df[(df['Equipe 1'] == equipe1) & (df['Equipe 2'] == equipe2)]['Résultat'].value_counts(),fill_value=0)[0]))
    except Exception as e:
        c = '0'
    df_match1['Historique'] = a + 'V ' + b + 'D ' + c + 'N'
    df_match2['Historique'] = b + 'V ' + a + 'D ' + c + 'N'

    df_match1.columns = noms_colonnes1
    df_match2.columns = noms_colonnes2

    for i, (col_name, value) in enumerate(df_match1.iloc[0].items()):
        if col_name == 'Classement':
            value = int(value)
        label_title = tk.Label(frame_tableau1, text=f"{col_name}:", padx=10, pady=5, borderwidth=1, relief="solid", anchor="w", width=30)
        label_title.grid(row=i+3, column=0, sticky="w")

        label_value = tk.Label(frame_tableau1, text=value, padx=10, pady=5, borderwidth=1, relief="solid", anchor="w", width=20)
        label_value.grid(row=i+3, column=1, sticky="w")
    
    for i, (col_name, value) in enumerate(df_match2.iloc[0].items()):
        if col_name == 'Classement':
            value = int(value)
        label_title = tk.Label(frame_tableau2, text=f"{col_name}:", padx=10, pady=5, borderwidth=1, relief="solid", anchor="w", width=30)
        label_title.grid(row=i+3, column=0, sticky="w")

        label_value = tk.Label(frame_tableau2, text=value, padx=10, pady=5, borderwidth=1, relief="solid", anchor="w", width=20)
        label_value.grid(row=i+3, column=1, sticky="w")

        image_path1 = trouver_chemins_images_avec_mot_cle('Logos\\', equipe1)
        image1 = Image.open(image_path1)
        image1 = image1.resize((100, 120), Image.ANTIALIAS)  
        photo1 = ImageTk.PhotoImage(image1)

        label_image1 = tk.Label(frame_tableau1, image=photo1)
        label_image1.image = photo1  
        label_image1.grid(row=0, column=0, columnspan=2)

        image_path2 = trouver_chemins_images_avec_mot_cle('Logos\\', equipe2)
        image2 = Image.open(image_path2)
        image2 = image2.resize((100, 120), Image.ANTIALIAS)  
        photo2 = ImageTk.PhotoImage(image2)

        label_image2 = tk.Label(frame_tableau2, image=photo2)
        label_image2.image = photo2  
        label_image2.grid(row=0, column=0, columnspan=2)

        bouton_predictions = tk.Button(window, text="Prédictions", command=lambda: fonction_predictions(event = None))
        bouton_predictions.place(relx=0.5, rely=0.8, anchor="center")


def fonction_predictions(event):
    global photo1, photo2, equipe1, equipe2

    new_window = tk.Toplevel()
    new_window.attributes('-fullscreen', True)
    titre_label = tk.Label(new_window, text="Prédiction pour le match " + equipe1 + " - " + equipe2, font=("Helvetica", 24, "bold"))
    titre_label.pack(pady=150)

    frame_tableau3 = tk.Frame(new_window)
    frame_tableau3.pack(expand=True, anchor="w")
    frame_tableau4 = tk.Frame(new_window)
    frame_tableau4.pack(expand=True, anchor="w")


    label_image1 = tk.Label(frame_tableau3, image=photo1)
    label_image1.image = photo1  
    label_image1.grid(row=0, column=0, columnspan=2)

    label_image2 = tk.Label(frame_tableau4, image=photo2)
    label_image2.image = photo2  
    label_image2.grid(row=0, column=0, columnspan=2)

    frame_tableau3.place(in_=new_window, anchor="w", relx=0.37, rely=0.4)
    frame_tableau4.place(in_=new_window, anchor="w", relx=0.57, rely=0.4)

    try:
        match_aller = df2[((df2['Equipe 1'] == equipe1) & (df2['Equipe 2'] == equipe2))][ "Historique"].iloc[0]
    except Exception as e:
        match_aller = 0

    df['Classement Equipe 1'] = 1/df['Classement Equipe 1']
    df['Classement Equipe 2'] = 1/df['Classement Equipe 2']
    X_test = df[((df['Equipe 1'] == equipe1) & (df['Journée'] == first))].drop(['Equipe 1', 'Equipe 2', 'Journée', 'Equipe 1 à Domicile', 'Buts Equipe 1', 'Buts Equipe 2', 'Résultat', 'Historique'], axis=1)
    X_test = X_test.reset_index(drop=True)
    X_test[["Classement Equipe 2","Moyenne_BM par 2", "Moyenne_BM par 2 à Domicile", "Moyenne_BM par 2 à Extérieur", "Moyenne_BE par 2", "Moyenne_BE par 2 à Domicile", "Moyenne_BE par 2 à Extérieur", 'Forme 2']] = (
        df[((df['Equipe 2'] == equipe2) & (df['Journée'] == first))][["Classement Equipe 2","Moyenne_BM par 2", "Moyenne_BM par 2 à Domicile","Moyenne_BM par 2 à Extérieur", "Moyenne_BE par 2", "Moyenne_BE par 2 à Domicile", "Moyenne_BE par 2 à Extérieur",'Forme 2']]
        .reset_index(drop=True)
    )
    X_test.insert(0, 'Equipe 1 à Domicile', 1)
    X_test['Historique'] = df2[((df2['Equipe 1'] == equipe1) & (df2['Equipe 2'] == equipe2))][ "Historique"].iloc[-1] + match_aller
    X_test[['BM Equipe 1 à D si D', 'BM Equipe 1 à E si E', 'BE Equipe 1 à D si D', 'BE Equipe 1 à E si E', 'BM Equipe 2 à D si D',
                      'BM Equipe 2 à E si E', 'BE Equipe 2 à D si D', 'BE Equipe 2 à E si E']] = X_test.apply(
        lambda row: [row['Equipe 1 à Domicile'] * row['Moyenne_BM par 1 à Domicile'], (1 - row['Equipe 1 à Domicile']) * row['Moyenne_BM par 1 à Extérieur'], 
                     row['Equipe 1 à Domicile'] * row['Moyenne_BE par 1 à Domicile'], (1 - row['Equipe 1 à Domicile']) * row['Moyenne_BE par 1 à Extérieur'],
                     (1-row['Equipe 1 à Domicile']) * row['Moyenne_BM par 2 à Domicile'], (row['Equipe 1 à Domicile']) * row['Moyenne_BM par 2 à Extérieur'], 
                     (1-row['Equipe 1 à Domicile']) * row['Moyenne_BE par 1 à Domicile'], (row['Equipe 1 à Domicile']) * row['Moyenne_BE par 1 à Extérieur']], axis=1, result_type='expand')
    
    X_test2 = df[((df['Equipe 1'] == equipe2) & (df['Journée'] == first))].drop(['Equipe 1', 'Equipe 2', 'Journée', 'Equipe 1 à Domicile', 'Buts Equipe 1', 'Buts Equipe 2', 'Résultat', 'Historique'], axis=1)
    X_test2 = X_test2.reset_index(drop=True)
    X_test2[["Classement Equipe 2","Moyenne_BM par 2", "Moyenne_BM par 2 à Domicile", "Moyenne_BM par 2 à Extérieur", "Moyenne_BE par 2", "Moyenne_BE par 2 à Domicile", "Moyenne_BE par 2 à Extérieur", 'Forme 2']] = (
        df[((df['Equipe 2'] == equipe1) & (df['Journée'] == first))][["Classement Equipe 2","Moyenne_BM par 2", "Moyenne_BM par 2 à Domicile","Moyenne_BM par 2 à Extérieur", "Moyenne_BE par 2", "Moyenne_BE par 2 à Domicile", "Moyenne_BE par 2 à Extérieur",'Forme 2']]
        .reset_index(drop=True)
    )
    X_test2.insert(0, 'Equipe 1 à Domicile', 0)
    X_test2['Historique'] = df2[((df2['Equipe 1'] == equipe2) & (df2['Equipe 2'] == equipe1))][ "Historique"].iloc[-1] - match_aller
    X_test2[['BM Equipe 1 à D si D', 'BM Equipe 1 à E si E', 'BE Equipe 1 à D si D', 'BE Equipe 1 à E si E', 'BM Equipe 2 à D si D',
                      'BM Equipe 2 à E si E', 'BE Equipe 2 à D si D', 'BE Equipe 2 à E si E']] = X_test2.apply(
        lambda row: [row['Equipe 1 à Domicile'] * row['Moyenne_BM par 1 à Domicile'], (1 - row['Equipe 1 à Domicile']) * row['Moyenne_BM par 1 à Extérieur'], 
                     row['Equipe 1 à Domicile'] * row['Moyenne_BE par 1 à Domicile'], (1 - row['Equipe 1 à Domicile']) * row['Moyenne_BE par 1 à Extérieur'],
                     (1-row['Equipe 1 à Domicile']) * row['Moyenne_BM par 2 à Domicile'], (row['Equipe 1 à Domicile']) * row['Moyenne_BM par 2 à Extérieur'], 
                     (1-row['Equipe 1 à Domicile']) * row['Moyenne_BE par 1 à Domicile'], (row['Equipe 1 à Domicile']) * row['Moyenne_BE par 1 à Extérieur']], axis=1, result_type='expand')

    X_test_sm1 = sm.add_constant(X_test, has_constant='add')
    X_test_sm2 = sm.add_constant(X_test2, has_constant='add')
    probabilites_buts1 = random_forest_model2.predict_proba(X_test)
    probabilites_buts2 = random_forest_model2.predict_proba(X_test2)
    
    y_pred1 = results2.predict(X_test_sm1)
    y_pred1 = [-1 if x < 0 else 1 for x in y_pred1]
    y_pred_int1 = results1.predict(X_test_sm1)
    y_pred_int2 = results1.predict(X_test_sm2)
    resultat = [a - b for a, b in zip(y_pred_int1, y_pred_int2)]
    y_pred2 = [-1 if x < 0 else 1 for x in  resultat]   
    y_pred3 = random_forest_model.predict(X_test)
    y_pred4 = svm_model.predict(X_test)

    poids_modele1 = 0.5
    poids_modele2 = 0.5
    poids_modele3 = 1
    poids_modele4 = 1

    resultat_final_pondere = poids_modele1 * y_pred1[0] + poids_modele2 * y_pred2[0] + poids_modele3 * y_pred3[0] + poids_modele4 * y_pred4[0]
    if resultat_final_pondere > 0.5:
        resultat_final = 1
    elif resultat_final_pondere < -0.5:
        resultat_final = -1
    else:
        resultat_final = 0

    print(y_pred_int1)
    print(y_pred_int2)
    print(y_pred1)
    print(y_pred2)
    print(y_pred3)
    print(y_pred4)
    print(resultat_final)
    if resultat_final == 1:
        while round(y_pred_int1[0]) - round(y_pred_int2[0]) < resultat_final:
            y_pred_int1[0] = y_pred_int1[0] + 1
    elif resultat_final == -1:
        while round(y_pred2[0]) - round(y_pred1[0]) < abs(resultat_final):
            y_pred_int2[0] = y_pred_int2[0] + 1
    else:
        if round(y_pred_int1[0]) - round(y_pred_int2[0]) < resultat_final:
            y_pred_int2[0] = y_pred_int1[0] 
        elif round(y_pred_int2[0]) - round(y_pred_int1[0]) < resultat_final:
            y_pred_int1[0] = y_pred_int2[0]

    probabilites_buts1 = list(enumerate(probabilites_buts1[0]))
    tableau1 = ttk.Treeview(new_window, columns=("nombres de buts", "proba"), show="headings")

    tableau1.heading("nombres de buts", text="Nombres de Buts",  anchor="w")
    tableau1.heading("proba", text="Probabilité",  anchor="w")

    for k in probabilites_buts1:
        tableau1.insert("", "end", values=k)
    
    tableau1.column("proba", width=100)
    tableau1.column("nombres de buts", width=100)

    probabilites_buts2 = list(enumerate(probabilites_buts2[0]))
    tableau2 = ttk.Treeview(new_window, columns=("nombres de buts", "proba"), show="headings")

    tableau2.heading("nombres de buts", text="Nombres de Buts",  anchor="w")
    tableau2.heading("proba", text="Probabilité",  anchor="w")

    for k in probabilites_buts2:
        tableau2.insert("", "end", values=k)

    tableau2.column("proba", width=100)
    tableau2.column("nombres de buts", width=100)
   
    tableau1.place(in_=new_window, anchor="w", relx=0.35, rely=0.6)
    tableau2.place(in_=new_window, anchor="w", relx=0.55, rely=0.6)

    score_1 = tk.Label(new_window, text = str(round(y_pred_int1[0])), font=("Helvetica", 24, "bold"))
    score_1.place(in_=new_window, anchor="w", relx=0.47, rely=0.4)
    
    score_2 = tk.Label(new_window, text = str(round(y_pred_int2[0])), font=("Helvetica", 24, "bold"))
    score_2.place(in_=new_window, anchor="w", relx=0.52, rely=0.4)

    tiret = tk.Label(new_window, text = '-', font=("Helvetica", 24, "bold"))
    tiret.place(in_=new_window, anchor="w", relx=0.49, rely=0.4)



    

window = tk.Tk()
window.title("Sélection de la Journée")
window.attributes('-fullscreen', True)


image_intro = Image.open('Logos\\Logo_Ligue_1_Uber_Eats_2022.png')
image_intro = image_intro.resize((130, 180), Image.ANTIALIAS)
photo_intro = ImageTk.PhotoImage(image_intro)



frame_intro = tk.Frame(window)
frame_intro.pack(expand=True, anchor="w")

label_image_intro = tk.Label(frame_intro, image=photo_intro)
label_image_intro.image = photo_intro
label_image_intro.grid(row=0, column=0)


frame_menus = tk.Frame(window)
frame_menus.pack(expand=True, anchor="w")

label_journee = tk.Label(frame_menus, text="Choisissez une journée:")
label_journee.grid(row=0, column=0, padx=10, pady=10)

journées = [f"J{i}" for i in range(first, 35)]
journées_combobox = ttk.Combobox(frame_menus, values=journées, state="readonly")
journées_combobox.set("Sélectionnez une journée")
journées_combobox.grid(row=0, column=1, padx=10, pady=10)
journées_combobox.bind("<<ComboboxSelected>>", fonction_journées)

label_match = tk.Label(frame_menus, text="Choisissez un match:")
label_match.grid(row=1, column=0, padx=10, pady=10)

matchs_combobox = ttk.Combobox(frame_menus, state="readonly")
matchs_combobox.set("Sélectionnez un match")
matchs_combobox.grid(row=1, column=1, padx=10, pady=10)
matchs_combobox.bind("<<ComboboxSelected>>", fonction_stats)


frame_tableau1 = tk.Frame(window)
frame_tableau1.pack(expand=True, anchor="w")
frame_tableau2 = tk.Frame(window)
frame_tableau2.pack(expand=True, anchor="w")



label_texte = tk.Label(frame_tableau1, text="")
label_texte.grid(row=2, column=0, columnspan=2, padx=10, pady=10, sticky="w")
label_texte = tk.Label(frame_tableau2, text="")
label_texte.grid(row=2, column=0, columnspan=2, padx=10, pady=10, sticky="w")

window.update_idletasks()
frame_intro.place(in_=window, anchor="w", relx=0.47, rely=0.1)
frame_menus.place(in_=window, anchor="w", relx=0.4, rely=0.25)
frame_tableau1.place(in_=window, anchor="w", relx=0.17, rely=0.4)
frame_tableau2.place(in_=window, anchor="w", relx=0.60, rely=0.4)




window.mainloop()








C:\Users\vtgra\AppData\Local\Temp\ipykernel_99212\1476549191.py:272: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image_intro = image_intro.resize((130, 180), Image.ANTIALIAS)


Chemin de l'image trouvée : Logos\Logo_Le Havre_AC_2012.svg.png
Chemin de l'image trouvée : Logos\Logo_Olympique_Lyonnais_-_2022.png
Chemin de l'image trouvée : Logos\Logo_Le Havre_AC_2012.svg.png
Chemin de l'image trouvée : Logos\Logo_Olympique_Lyonnais_-_2022.png
Chemin de l'image trouvée : Logos\Logo_Le Havre_AC_2012.svg.png
Chemin de l'image trouvée : Logos\Logo_Olympique_Lyonnais_-_2022.png
Chemin de l'image trouvée : Logos\Logo_Le Havre_AC_2012.svg.png
Chemin de l'image trouvée : Logos\Logo_Olympique_Lyonnais_-_2022.png
Chemin de l'image trouvée : Logos\Logo_Le Havre_AC_2012.svg.png
Chemin de l'image trouvée : Logos\Logo_Olympique_Lyonnais_-_2022.png
Chemin de l'image trouvée : Logos\Logo_Le Havre_AC_2012.svg.png
Chemin de l'image trouvée : Logos\Logo_Olympique_Lyonnais_-_2022.png


C:\Users\vtgra\AppData\Local\Temp\ipykernel_99212\1476549191.py:98: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image1 = image1.resize((100, 120), Image.ANTIALIAS)
C:\Users\vtgra\AppData\Local\Temp\ipykernel_99212\1476549191.py:107: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image2 = image2.resize((100, 120), Image.ANTIALIAS)


Chemin de l'image trouvée : Logos\Logo_Le Havre_AC_2012.svg.png
Chemin de l'image trouvée : Logos\Logo_Olympique_Lyonnais_-_2022.png
0    1.410747
dtype: object
0    0.904856
dtype: object
[1]
[1]
[1]
[1]
1
Chemin de l'image trouvée : Logos\Logo_RC_Lens_2014.svg.png
Chemin de l'image trouvée : Logos\Paris S-G_Logo.svg.png
Chemin de l'image trouvée : Logos\Logo_RC_Lens_2014.svg.png
Chemin de l'image trouvée : Logos\Paris S-G_Logo.svg.png
Chemin de l'image trouvée : Logos\Logo_RC_Lens_2014.svg.png
Chemin de l'image trouvée : Logos\Paris S-G_Logo.svg.png
Chemin de l'image trouvée : Logos\Logo_RC_Lens_2014.svg.png
Chemin de l'image trouvée : Logos\Paris S-G_Logo.svg.png
Chemin de l'image trouvée : Logos\Logo_RC_Lens_2014.svg.png
Chemin de l'image trouvée : Logos\Paris S-G_Logo.svg.png
Chemin de l'image trouvée : Logos\Logo_RC_Lens_2014.svg.png
Chemin de l'image trouvée : Logos\Paris S-G_Logo.svg.png
Chemin de l'image trouvée : Logos\Logo_RC_Lens_2014.svg.png
Chemin de l'image trouvée : Log

C:\Users\vtgra\AppData\Local\Temp\ipykernel_99212\1476549191.py:98: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image1 = image1.resize((100, 120), Image.ANTIALIAS)
C:\Users\vtgra\AppData\Local\Temp\ipykernel_99212\1476549191.py:107: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image2 = image2.resize((100, 120), Image.ANTIALIAS)


0    2.667578
dtype: object
0    1.173059
dtype: object
[1]
[1]
[-1]
[0]
0
Chemin de l'image trouvée : Logos\Logo_FC_Nantes_(avec_fond)_-_2019.png
Chemin de l'image trouvée : Logos\Logo_Clermont Foot_63_depuis_2020.png
Chemin de l'image trouvée : Logos\Logo_FC_Nantes_(avec_fond)_-_2019.png
Chemin de l'image trouvée : Logos\Logo_Clermont Foot_63_depuis_2020.png
Chemin de l'image trouvée : Logos\Logo_FC_Nantes_(avec_fond)_-_2019.png
Chemin de l'image trouvée : Logos\Logo_Clermont Foot_63_depuis_2020.png
Chemin de l'image trouvée : Logos\Logo_FC_Nantes_(avec_fond)_-_2019.png
Chemin de l'image trouvée : Logos\Logo_Clermont Foot_63_depuis_2020.png
Chemin de l'image trouvée : Logos\Logo_FC_Nantes_(avec_fond)_-_2019.png
Chemin de l'image trouvée : Logos\Logo_Clermont Foot_63_depuis_2020.png
Chemin de l'image trouvée : Logos\Logo_FC_Nantes_(avec_fond)_-_2019.png
Chemin de l'image trouvée : Logos\Logo_Clermont Foot_63_depuis_2020.png
Chemin de l'image trouvée : Logos\Logo_FC_Nantes_(avec_fond)_

C:\Users\vtgra\AppData\Local\Temp\ipykernel_99212\1476549191.py:98: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image1 = image1.resize((100, 120), Image.ANTIALIAS)
C:\Users\vtgra\AppData\Local\Temp\ipykernel_99212\1476549191.py:107: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  image2 = image2.resize((100, 120), Image.ANTIALIAS)


0    1.559379
dtype: object
0    0.910831
dtype: object
[1]
[1]
[1]
[1]
1
